In [53]:
import numpy as np
import requests
import wikipedia

In [16]:
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

TITLE = 'Jurassic Park (novel)'

PARAMS = {
    'action': "query",
    'titles': TITLE,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
data = R.json()
#print(data['query']['title'])

In [29]:
data['query']['pages'][list(data['query']['pages'])[0]]['links'][:20]

[{'ns': 0, 'title': '2000 AD (comics)'},
 {'ns': 0, 'title': 'A Case of Need'},
 {'ns': 0, 'title': 'Airframe (novel)'},
 {'ns': 0, 'title': 'Alfred A. Knopf'},
 {'ns': 0, 'title': 'Amazon (video game)'},
 {'ns': 0, 'title': 'Amber'},
 {'ns': 0, 'title': 'Amphibian'},
 {'ns': 0, 'title': 'Amusement park'},
 {'ns': 0, 'title': 'Ancient DNA'},
 {'ns': 0, 'title': 'Andrew Ferguson'},
 {'ns': 0, 'title': 'Auxotrophy'},
 {'ns': 0, 'title': 'BILBY Award'},
 {'ns': 0, 'title': 'Backdoor (computing)'},
 {'ns': 0, 'title': 'Barnes & Noble'},
 {'ns': 0, 'title': 'Battle at Big Rock'},
 {'ns': 0, 'title': 'Beyond Westworld'},
 {'ns': 0, 'title': 'Binary (novel)'},
 {'ns': 0, 'title': 'Biotechnology'},
 {'ns': 0, 'title': 'Bird'},
 {'ns': 0, 'title': 'Canopy Flyer'}]

In [30]:
links = data['query']['pages'][list(data['query']['pages'])[0]]['links']

In [38]:
n = np.random.randint(0,len(links))
links[n]

{'ns': 0, 'title': 'DNA'}

In [46]:
n = np.random.randint(0,len(links))
NEXT = links[n]['title']

PARAMS = {
    'action': "query",
    'titles': NEXT,
    'prop': "links",
    'pllimit': "max",
    'format': "json",
}

R = S.get(url=URL, params=PARAMS)
next_data = R.json()
print(links[n]['title'])
print(next_data['query']['pages'][list(next_data['query']['pages'])[0]]['title'])

Jurassic World: Original Motion Picture Soundtrack
Jurassic World: Original Motion Picture Soundtrack


In [48]:
def get_links(TITLE):
    PARAMS = {
        'action': "query",
        'titles': TITLE,
        'prop': "links",
        'pllimit': "max",
        'format': "json",
    }

    R = S.get(url=URL, params=PARAMS)
    return R.json()

In [52]:
start = input()
hops = input()

print("0 : " + start)
title = start
for i in range(int(hops)):
    response = get_links(title)
    links = response['query']['pages'][list(response['query']['pages'])[0]]['links']
    n = np.random.randint(0,len(links))
    title = links[n]['title']
    print(str(i+1) + " : " + title)

 Jurassic Park (novel)
 5


0 : Jurassic Park (novel)
1 : Dragon curve
2 : Blancmange curve
3 : Archimedes
4 : Menander
5 : Akrai


In [54]:
jp = wikipedia.page('Jurassic Park (novel)')

In [63]:
import nltk
from nltk.corpus import stopwords
import nltk.collocations
from nltk import FreqDist, word_tokenize
import string
import re

In [70]:
def get_50_most_common(page):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    tokens_raw = nltk.regexp_tokenize(page.content, pattern)
    tokens = [word.lower() for word in tokens_raw]

    stopwords_list = stopwords.words('english')
    stopwords_list += list(string.punctuation)
    stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

    words_stopped = [word for word in tokens if word not in stopwords_list]

    freqdist = FreqDist(words_stopped)
    return freqdist.most_common(50)

In [72]:
get_50_most_common(wikipedia.page('Catwoman'))

[('batman', 179),
 ('catwoman', 155),
 ('selina', 142),
 ('bruce', 50),
 ('two', 32),
 ('vol', 30),
 ('series', 29),
 ('story', 29),
 ('kyle', 28),
 ('one', 28),
 ('later', 28),
 ("selina's", 28),
 ('new', 28),
 ('gotham', 27),
 ('cat', 26),
 ('wayne', 25),
 ('comics', 24),
 ('time', 24),
 ('also', 24),
 ('relationship', 21),
 ('dc', 20),
 ('however', 20),
 ('earth', 20),
 ('city', 19),
 ('costume', 19),
 ('daughter', 19),
 ('joker', 17),
 ('black', 17),
 ('character', 16),
 ('mask', 16),
 ('first', 15),
 ('revealed', 15),
 ('end', 15),
 ('hush', 15),
 ('comic', 14),
 ("batman's", 14),
 ('holly', 14),
 ('life', 14),
 ('storyline', 14),
 ('ivy', 14),
 ('appears', 13),
 ('identity', 13),
 ('film', 13),
 ('returns', 13),
 ('years', 13),
 ('zatanna', 13),
 ('although', 11),
 ('romantic', 11),
 ('helena', 11),
 ('maggie', 11)]